# Scope del Proyecto

## Objetivo general:

* Analizar la satisfacción de los pasajeros de las aerolíneas y determinar los factores que tienen el mayor impacto en su satisfacción.

## Objetivos específicos:

* Identificar las variables que están más correlacionadas con la satisfacción de los pasajeros.
* Analizar la relación entre la satisfacción de los pasajeros y los diferentes aspectos del servicio de la aerolínea, como la comodidad del asiento, el entretenimiento a bordo, la limpieza, etc.
* Determinar si hay diferencias significativas en la satisfacción de los pasajeros según la aerolínea, el tipo de viaje (nacional/internacional), la clase de viaje, el destino, etc.
* Identificar patrones en la satisfacción de los pasajeros a lo largo del tiempo.

## Preguntas Planteadas
1. ¿Cuantos clientes leales tuvieron vuelos satisfactorios?
2. ¿A qué sexo (masculino y femenino) le costó más agendar un vuelo en línea?
3. ¿Cuáles son los factores que tienen el mayor impacto en la satisfacción de los pasajeros?
4. ¿Hay alguna diferencia en la satisfacción de los pasajeros entre las diferentes aerolíneas?
5. ¿La satisfacción de los pasajeros difiere según el tipo de viaje o la clase de viaje?

# Exploración de datos

In [1]:
# Importación de librerias utilizadas en el proyecto
import numpy as np
import pandas as pd
import boto3
import psycopg2
import configparser
import io
from io import BytesIO
import random
from sqlalchemy import create_engine

In [2]:
config = configparser.ConfigParser()
config.read('escec.cfg')

['escec.cfg']

In [3]:
# Se prepara para la lectura del dataset almacenado en S3
s3 = boto3.resource(
    service_name = 's3',
    region_name = 'us-east-1',
    aws_access_key_id = config.get('IAM', 'ACCESS_KEY'),
    aws_secret_access_key = config.get('IAM', 'SECRET_ACCESS_KEY')
)

In [4]:
for bucket in s3.buckets.all():
    S3_BUCKET_NAME = bucket.name
    print(bucket.name)

airlineairports
airlinesatisfactiondataset


In [5]:
S3_BUCKET_AIRLINES = 'airlinesatisfactiondataset'
FILE_NAME = 'data/airline_satisfaction.csv'

In [6]:
S3_BUCKET_AIRLPORTS = 'airlineairports'
FILE_NAME = 'data/airports.csv'

In [7]:
#extraemos todo lo que está en el bucket para los datos de satisfaccion
remoteFileListAirline = []
for objt in s3.Bucket(S3_BUCKET_AIRLINES).objects.all():
    remoteFileListAirline.append(objt.key)

remoteFileListAirline

['data/', 'data/airline_satisfaction.csv']

In [8]:
#extraemos todo lo que está en el bucket para los datos de aeropuertos
remoteFileListAirports = []
for objt in s3.Bucket(S3_BUCKET_AIRLPORTS).objects.all():
    remoteFileListAirports.append(objt.key)

remoteFileListAirports

['data/', 'data/airports.csv']

## Se lee el archivo con datos de satisfacción de los aeropuertos

In [9]:
for remoteFile in remoteFileListAirports:
    try:
        file = s3.Bucket(S3_BUCKET_AIRLPORTS).Object(remoteFile).get()
        data = file['Body'].read()
        df_airports = pd.read_csv(BytesIO(data))
    except Exception as ex:
        print("No es un archivo.")
        print(ex)

df_airports.head()

No es un archivo.
No columns to parse from file


,idAirport,airportName,airportCode
0,1,Ukhta Airport,UCT
1,2,Aleknagik / New Airport,WKK
2,3,Guangzhou Baiyun International Airport,CAN
3,4,Leipzig/Halle Airport,LEJ
4,5,Charleston Air Force Base-International Airport,CHS


## Se lee el archivo con datos de satisfacción de las aerolineas

In [10]:
for remoteFile in remoteFileListAirline:
    try:
        file = s3.Bucket(S3_BUCKET_AIRLINES).Object(remoteFile).get()
        data = file['Body'].read()
        df = pd.read_csv(BytesIO(data))
        # Se elimina la primera columna del dataframe (era el número de fila)
        del df[df.columns[0]]
    except Exception as ex:
        print("No es un archivo.")
        print(ex)

df.head()

No es un archivo.
No columns to parse from file


,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction,idAirport;;;;;
0,106966,Female,Loyal Customer,40,Business travel,Eco,1036,2,3,3,...,1,1,2,1,2,2,26,32.0,neutral or dissatisfied,105;;;;;
1,54152,Female,Loyal Customer,34,Business travel,Business,1569,2,2,2,...,3,4,3,3,3,3,0,0.0,satisfied,931;;;;;
2,26463,Female,Loyal Customer,54,Personal Travel,Eco,925,3,4,4,...,4,4,4,3,4,2,0,1.0,neutral or dissatisfied,800;;;;;
3,46257,Female,Loyal Customer,69,Personal Travel,Eco,624,3,1,3,...,4,3,4,1,4,3,36,33.0,neutral or dissatisfied,986;;;;;
4,33133,Male,Loyal Customer,48,Personal Travel,Eco,102,4,4,4,...,4,3,5,5,5,1,1,4.0,satisfied,801;;;;;


In [11]:
# Se limpian los datos con caracteres extraños

df['idAirport;;;;;']# limpiar la columna "idAirport" eliminando los caracteres ";"
df['idAirport;;;;;'] = df['idAirport;;;;;'].str.replace(';', '')

# cambiar el nombre de la columna "idAirport" a "idAirport"
df.columns = ['idAirport' if x=='idAirport;;;;;' else x for x in df.columns]

# Los valores de delay nulos se vuelven 0's
df['Departure Delay in Minutes'] = df['Departure Delay in Minutes'].fillna(0)
df['Arrival Delay in Minutes'] = df['Arrival Delay in Minutes'].fillna(0)

df.head()

,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction,idAirport
0,106966,Female,Loyal Customer,40,Business travel,Eco,1036,2,3,3,...,1,1,2,1,2,2,26,32.0,neutral or dissatisfied,105
1,54152,Female,Loyal Customer,34,Business travel,Business,1569,2,2,2,...,3,4,3,3,3,3,0,0.0,satisfied,931
2,26463,Female,Loyal Customer,54,Personal Travel,Eco,925,3,4,4,...,4,4,4,3,4,2,0,1.0,neutral or dissatisfied,800
3,46257,Female,Loyal Customer,69,Personal Travel,Eco,624,3,1,3,...,4,3,4,1,4,3,36,33.0,neutral or dissatisfied,986
4,33133,Male,Loyal Customer,48,Personal Travel,Eco,102,4,4,4,...,4,3,5,5,5,1,1,4.0,satisfied,801


## Se unen las distintas fuentes de información

In [12]:
# Obtiene una lista de los idAirport del dataset de aeropuertos
# airport_ids = list(df_airports['idAirport'])

# Agrega una nueva columna "idAirport" al dataset de satisfacción y llena cada fila con un idAirport aleatorio
# df['idAirport'] = [random.choice(airport_ids) for _ in range(len(df))]

In [13]:
# Dataset de satisfacción con la nueva columna
# df.head()

In [14]:
# guardar el DataFrame como un archivo CSV
# df.to_csv('airline_satisfaction.csv', index=False)

## Análisis exploratorio del dataset de satisfacción

In [15]:
# Se obtienen estadísticas descriptivas de variables numéricas
print(df.describe())

                  id           Age  Flight Distance  Inflight wifi service  \
count   39999.000000  39999.000000     39999.000000           39999.000000   
mean    64883.149154     39.393060      1191.004425               2.728768   
std     37471.739749     15.094959       994.866002               1.326630   
min         2.000000      7.000000        31.000000               0.000000   
25%     32485.500000     27.000000       416.000000               2.000000   
50%     64951.000000     40.000000       846.000000               3.000000   
75%     97285.500000     51.000000      1739.000000               4.000000   
max    129880.000000     85.000000      4983.000000               5.000000   

       Departure/Arrival time convenient  Ease of Online booking  \
count                       39999.000000            39999.000000   
mean                            3.054601                2.754969   
std                             1.521310                1.396526   
min                      

In [16]:
# Se obtiene la distribución de los valores en la variable "satisfaction"
print(df["satisfaction"].value_counts())

neutral or dissatisfied    22732
satisfied                  17267
Name: satisfaction, dtype: int64


In [17]:
# Se calcula la correlación entre variables numéricas
print(df.corr())

                                         id       Age  Flight Distance  \
id                                 1.000000  0.018632         0.092981   
Age                                0.018632  1.000000         0.103459   
Flight Distance                    0.092981  0.103459         1.000000   
Inflight wifi service             -0.022387  0.015945         0.003062   
Departure/Arrival time convenient -0.000570  0.041585        -0.022547   
Ease of Online booking             0.014507  0.021882         0.060478   
Gate location                      0.003063 -0.001064         0.002079   
Food and drink                    -0.003640  0.022470         0.052078   
Online boarding                    0.056004  0.209083         0.215556   
Seat comfort                       0.044269  0.156752         0.158683   
Inflight entertainment            -0.006200  0.074431         0.123347   
On-board service                   0.051124  0.054516         0.111838   
Leg room service                   0.0

# Modelo Dimensional Propuesto

### Tabla de Hechos (Satisfacción del Cliente)

* idCliente (clave foránea)
* idViaje (clave foránea)
* idServicio (clave foránea)
* idAeropuerto (clave foránea)
* satisfaction
* DepartureDelay
* ArrivalDelay

### Dimensión Cliente

* IdCliente (clave primaria)
* Gender
* CustomerType
* age

### Dimensión Viaje

* idViaje (clave primaria)
* TypeTravel
* Class
* FlightDistance

### Dimensión Servicio

* idServicio (clave primaria)
* InflightWifiService
* DepartureArrivalTimeConvenient
* EaseOnlineBooking
* GateLocation
* FoodAndDrink
* OnlineBoarding
* SeatComfort
* InflightEntertainment
* OnboardService
* LegRoomService
* BaggageHandling
* CheckinService
* InflightService
* Cleanliness

### Dimensión Aeropuerto

* idAeropuerto
* airportName
* airportCode

## Preparando la base de datos en RDS para la carga de las dimensiones
## y tabla de hechos

In [18]:
# Se crea una instancia de RDS
aws_conn = boto3.client('rds', aws_access_key_id=config.get('IAM', 'ACCESS_KEY'),
                    aws_secret_access_key=config.get('IAM', 'SECRET_ACCESS_KEY'),
                    region_name='us-east-1')

In [19]:
rdsIdentifier = 'al-satisfaction-db' #nombre de la instancia

In [20]:
# Se crea una instancia de base de datos en RDS (Se comenta porque solamente se utiliza una vez)
""" try:
    response = aws_conn.create_db_instance(
            AllocatedStorage=10,
            DBName=config.get('RDS', 'DB_NAME'),
            DBInstanceIdentifier=rdsIdentifier,
            DBInstanceClass="db.t3.micro",
            Engine="postgres",
            MasterUsername=config.get('RDS', 'DB_USER'),
            MasterUserPassword=config.get('RDS', 'DB_PASSWORD'),
            Port=int(config.get('RDS', 'DB_PORT')),
            VpcSecurityGroupIds=[config.get('VPC', 'SECURITY_GROUP')],
            PubliclyAccessible=True
        )
    print(response)
except aws_conn.exceptions.DBInstanceAlreadyExistsFault as ex:
    print("La Instancia de Base de Datos ya Existe.") """

' try:\n    response = aws_conn.create_db_instance(\n            AllocatedStorage=10,\n            DBName=config.get(\'RDS\', \'DB_NAME\'),\n            DBInstanceIdentifier=rdsIdentifier,\n            DBInstanceClass="db.t3.micro",\n            Engine="postgres",\n            MasterUsername=config.get(\'RDS\', \'DB_USER\'),\n            MasterUserPassword=config.get(\'RDS\', \'DB_PASSWORD\'),\n            Port=int(config.get(\'RDS\', \'DB_PORT\')),\n            VpcSecurityGroupIds=[config.get(\'VPC\', \'SECURITY_GROUP\')],\n            PubliclyAccessible=True\n        )\n    print(response)\nexcept aws_conn.exceptions.DBInstanceAlreadyExistsFault as ex:\n    print("La Instancia de Base de Datos ya Existe.") '

In [21]:
rdsInstanceIds = []

response = aws_conn.describe_db_instances()
for resp in response['DBInstances']:
    rdsInstanceIds.append(resp['DBInstanceIdentifier'])
    db_instance_status = resp['DBInstanceStatus']

print(f"DBInstanceIds {rdsInstanceIds}")

DBInstanceIds ['al-satisfaction-db']


In [22]:
# Se obtiene el URL del host
try:
     instances = aws_conn.describe_db_instances(DBInstanceIdentifier=rdsIdentifier)
     RDS_HOST = instances.get('DBInstances')[0].get('Endpoint').get('Address')
     print(RDS_HOST)
except Exception as ex:
     print("La instancia de base de datos no existe o aun no se ha terminado de crear.")
     print(ex)

al-satisfaction-db.csxwhg4rwkek.us-east-1.rds.amazonaws.com


## Se agregan las tablas de las dimensiones y tabla de hechos del DDL

In [23]:
# Se comentan para que en la ejecución no vuelva a ejecutarse
'''import dw_ddl

try:
    db_conn = psycopg2.connect(
        database=config.get('RDS', 'DB_NAME'), 
        user=config.get('RDS', 'DB_USER'),
        password=config.get('RDS', 'DB_PASSWORD'), 
        host=RDS_HOST,
        port=config.get('RDS', 'DB_PORT')
    )

    cursor = db_conn.cursor()
    cursor.execute(dw_ddl.DDL_QUERY)
    db_conn.commit()
    print("Base de Datos Creada Exitosamente")
except Exception as ex:
    print("ERROR: Error al crear la base de datos.")
    print(ex) '''

'import dw_ddl\n\ntry:\n    db_conn = psycopg2.connect(\n        database=config.get(\'RDS\', \'DB_NAME\'), \n        user=config.get(\'RDS\', \'DB_USER\'),\n        password=config.get(\'RDS\', \'DB_PASSWORD\'), \n        host=RDS_HOST,\n        port=config.get(\'RDS\', \'DB_PORT\')\n    )\n\n    cursor = db_conn.cursor()\n    cursor.execute(dw_ddl.DDL_QUERY)\n    db_conn.commit()\n    print("Base de Datos Creada Exitosamente")\nexcept Exception as ex:\n    print("ERROR: Error al crear la base de datos.")\n    print(ex) '

## Se crea una función para insertar en las tablas

In [24]:
def insertDataToSQL(data_dict, table_name):
     postgres_driver = f"""postgresql://{config.get('RDS', 'DB_USER')}:{config.get('RDS', 'DB_PASSWORD')}@{RDS_HOST}:{config.get('RDS', 'DB_PORT')}/{config.get('RDS', 'DB_NAME')}"""    
     df_data = pd.DataFrame.from_records(data_dict)
     try:
          response = df_data.to_sql(table_name, postgres_driver, index=False, if_exists='append')
          print(f'Se han insertado {response} nuevos registros.' )
     except Exception as ex:
          print(ex)

## Se crean diccionarios para realizar los insert

In [25]:
# Se crea una lista de diccionarios para insertar en las tablas
dict_clientes = []
dict_servicios = []
dict_viajes = []
dict_viaje_satisfaccion = []

for i, row in df.iterrows():

    dict_fila_cliente = {
                'idcliente': i+1, 'gender': row['Gender'], 
                 'customer_type': row['Customer Type'], 'age': row['Age'] 
                 }
    
    dict_fila_servicio = {
                'idservicio': i+1, 'wifi_service': row['Inflight wifi service'], 
                 'ease_of_online_booking': row['Ease of Online booking'], 
                 'gate_location': row['Gate location'], 
                 'food_and_drink': row['Food and drink'], 
                 'online_boarding': row['Online boarding'], 
                 'seat_comfort': row['Seat comfort'], 
                 'inflight_entertainment': row['Inflight entertainment'], 
                 'on_board_service': row['On-board service'], 
                 'leg_room_service': row['Leg room service'], 
                 'baggage_handling': row['Baggage handling'], 
                 'checkin_service': row['Checkin service'], 
                 'inflight_service': row['Inflight service'], 
                 'cleanliness': row['Cleanliness'], 
                 }
    
    dict_fila_viaje = {
                'idviaje': row['id'], 'flight_distance': row['Flight Distance'], 
                 'travel_type': row['Type of Travel'], 'class': row['Class'] 
                 }
    
    dict_fila_viaje_satisfaccion = {
                'idcliente': i+1, 'idviaje': row['id'], 'idservicio': i+1,  
                'idaeropuerto': row['idAirport'], 'satisfaction': row['satisfaction'],
                'departuredelay': row['Departure Delay in Minutes'], 
                'arrivaldelay': row['Arrival Delay in Minutes'],
                }
    
    dict_clientes.append(dict_fila_cliente)
    dict_servicios.append(dict_fila_servicio)
    dict_viajes.append(dict_fila_viaje)
    dict_viaje_satisfaccion.append(dict_fila_viaje_satisfaccion)

In [26]:
# Le pegamos todos los aeropuertos del dataset de aeropuertos
dict_aeropuertos = []
for i, row in df_airports.iterrows():
    dict_fila_aeropuerto = {
                'idaeropuerto': row['idAirport'], 'airportname': row['airportName'], 
                 'airportcode': row['airportCode']
                 }
    dict_aeropuertos.append(dict_fila_aeropuerto)

## Se insertan los valores a la tabla dimensional de Cliente

In [27]:
# Se llama la función para insertar los valores en la tabla de cliente
# insertDataToSQL(dict_clientes, 'cliente')

## Se insertan los valores a la tabla dimensional de Viaje

In [28]:
# Se llama la función para insertar los valores en la tabla de viaje
# insertDataToSQL(dict_viajes, 'viaje')

## Se insertan los valores a la tabla dimensional de Servicio

In [29]:
# Se llama la función para insertar los valores en la tabla de servicio
# insertDataToSQL(dict_servicios, 'servicio')

## Se insertan los valores a la tabla dimensional de Aeropuerto

In [30]:
# Se llama la función para insertar los valores en la tabla de aeropuerto
# insertDataToSQL(dict_aeropuertos, 'aeropuerto')

## Se insertan los valores a la tabla de hechos Viaje Satisfacción

In [31]:
# Se llama la función para insertar los valores en la tabla de viaje
# insertDataToSQL(dict_viaje_satisfaccion, 'viaje_satisfaccion')

## Empezamos con las queries para traer la data de las tablas

In [32]:
# Llamamos al driver para conectarnos con la base de datos en postgresql
postgres_driver = f"""postgresql://{config.get('RDS', 'DB_USER')}:{config.get('RDS', 'DB_PASSWORD')}@{RDS_HOST}:{config.get('RDS', 'DB_PORT')}/{config.get('RDS', 'DB_NAME')}"""

### Obtenemos la dimensión de Cliente

In [33]:
sql_query = 'SELECT * FROM cliente;'
df_clientes = pd.read_sql(sql_query, postgres_driver)
df_clientes.head()

,idcliente,gender,customer_type,age
0,1,Female,Loyal Customer,40
1,2,Female,Loyal Customer,34
2,3,Female,Loyal Customer,54
3,4,Female,Loyal Customer,69
4,5,Male,Loyal Customer,48


### Obtenemos la dimensión de Servicio

In [34]:
sql_query = 'SELECT * FROM servicio;'
df_servicios = pd.read_sql(sql_query, postgres_driver)
df_servicios.head()

,idservicio,wifi_service,ease_of_online_booking,gate_location,food_and_drink,online_boarding,seat_comfort,inflight_entertainment,on_board_service,leg_room_service,baggage_handling,checkin_service,inflight_service,cleanliness
0,1,2,3,3,2,2,2,2,1,1,2,1,2,2
1,2,2,2,2,3,1,3,3,3,4,3,3,3,3
2,3,3,4,4,3,3,4,4,4,4,4,3,4,2
3,4,3,3,4,4,3,3,4,4,3,4,1,4,3
4,5,4,4,3,1,4,1,1,4,3,5,5,5,1


### Obtenemos la dimensión de Viaje

In [35]:
sql_query = 'SELECT * FROM viaje;'
df_viajes = pd.read_sql(sql_query, postgres_driver)
df_viajes.head()

,idviaje,flight_distance,travel_type,class
0,106966,1036,Business travel,Eco
1,54152,1569,Business travel,Business
2,26463,925,Personal Travel,Eco
3,46257,624,Personal Travel,Eco
4,33133,102,Personal Travel,Eco


### Obtenemos la dimensión de Aeropuerto

In [36]:
sql_query = 'SELECT * FROM aeropuerto;'
df_aeropuertos = pd.read_sql(sql_query, postgres_driver)
df_aeropuertos.head()

,idaeropuerto,airportname,airportcode
0,1,Ukhta Airport,UCT
1,2,Aleknagik / New Airport,WKK
2,3,Guangzhou Baiyun International Airport,CAN
3,4,Leipzig/Halle Airport,LEJ
4,5,Charleston Air Force Base-International Airport,CHS


### Obtenemos la tabla de hechos Viaje Satisfacción

In [38]:
sql_query = 'SELECT * FROM viaje_satisfaccion;'
df_viaje_satisfaccion = pd.read_sql(sql_query, postgres_driver)
df_viaje_satisfaccion.head()

,idcliente,idviaje,idservicio,idaeropuerto,satisfaction,departuredelay,arrivaldelay
0,1,106966,1,105,neutral or dissatisfied,26,32.0
1,2,54152,2,931,satisfied,0,0.0
2,3,26463,3,800,neutral or dissatisfied,0,1.0
3,4,46257,4,986,neutral or dissatisfied,36,33.0
4,5,33133,5,801,satisfied,1,4.0


## Empezamos a resolver las preguntas planteadas

#### 1. ¿Cuantos clientes leales tuvieron vuelos satisfactorios?

In [39]:
# Se filtran los clientes leales
df_clientes_leales = df_clientes[df_clientes['customer_type'] == 'Loyal Customer']

# Luego filtramos los vuelos satisfactorios
df_vuelos_satisfactorios = df_viaje_satisfaccion[df_viaje_satisfaccion['satisfaction'] == 'satisfied']

# Combinamos df_clientes_leales y df_vuelos_satisfactorios utilizando merge()
df_clientes_leales_satisfactorios = pd.merge(df_clientes_leales, df_vuelos_satisfactorios, on='idcliente')

# Contamos el número de clientes leales con vuelos satisfactorios
num_clientes_leales_satisfactorios = df_clientes_leales_satisfactorios['idcliente'].nunique()

print(f"El número de clientes leales con vuelos satisfactorios es: {num_clientes_leales_satisfactorios}")

El número de clientes leales con vuelos satisfactorios es: 15539


#### 2. ¿A qué sexo (masculino y femenino) le costó más agendar un vuelo en línea?

In [41]:
# Combinamos la tablas 'cliente' y 'viaje_satisfaccion'
clientes_viaje_satisfaccion = pd.merge(df_clientes, df_viaje_satisfaccion, on='idcliente')

# Luego combinamos las tablas 'clientes_viaje_satisfaccion' y 'servicio'
clientes_servicio = pd.merge(clientes_viaje_satisfaccion, df_servicios, on='idservicio')

# Calculamos la dificultad promedio de agendar un vuelo en línea para cada sexo
dificultad_promedio_por_sexo = clientes_servicio.groupby('gender')['ease_of_online_booking'].mean()

print(dificultad_promedio_por_sexo)
print("Por lo tanto, según los resultados le costó más a los hombres")

gender
Female    2.737814
Male      2.772557
Name: ease_of_online_booking, dtype: float64
Por lo tanto, según los resultados le costó más a los hombres


#### 3. ¿Cuáles son los factores que tienen el mayor impacto en la satisfacción de los pasajeros?

In [42]:
# Combinamos las tablas 'viaje_satisfaccion' y 'servicio'
satisfaccion_servicio = pd.merge(df_viaje_satisfaccion, df_servicios, on='idservicio')

# Combinamos la columna 'satisfaction' en valores numéricos (1 para 'satisfied' y 0 para 'neutral or dissatisfied')
satisfaccion_servicio['satisfaction_encoded'] = satisfaccion_servicio['satisfaction'].apply(lambda x: 1 if x == 'satisfied' else 0)

# Eliminamos la columna 'satisfaction' original para evitar errores de tipo
satisfaccion_servicio = satisfaccion_servicio.drop(columns=['satisfaction'])

# Calculamos las correlaciones entre la satisfacción de los pasajeros y cada uno de los factores de servicio
correlaciones = satisfaccion_servicio.corr(method='pearson')['satisfaction_encoded']

# Mostramos las correlaciones en orden descendente para ver qué factores tienen el mayor impacto en la satisfacción de los pasajeros
correlaciones_absolutas = correlaciones.abs()
correlaciones_ordenadas = correlaciones_absolutas.sort_values(ascending=False)

print(correlaciones_ordenadas)

satisfaction_encoded      1.000000
online_boarding           0.501286
inflight_entertainment    0.394350
seat_comfort              0.344905
on_board_service          0.320996
leg_room_service          0.310888
cleanliness               0.299173
wifi_service              0.280577
inflight_service          0.244741
baggage_handling          0.243502
checkin_service           0.238632
food_and_drink            0.202840
ease_of_online_booking    0.168680
arrivaldelay              0.056557
departuredelay            0.050529
idviaje                   0.014911
idaeropuerto              0.008857
gate_location             0.001299
idservicio                0.001115
idcliente                 0.001115
Name: satisfaction_encoded, dtype: float64


#### 4. ¿Hay alguna diferencia en la satisfacción de los pasajeros entre las diferentes aerolíneas?

In [43]:
# Combinamos las tablas 'viaje_satisfaccion' y 'aeropuerto'
satisfaccion_aeropuerto = pd.merge(df_viaje_satisfaccion, df_aeropuertos, left_on='idaeropuerto', right_on='idaeropuerto')

# Luego calculamos la proporción de pasajeros satisfechos para cada aerolínea
satisfaccion_por_aerolinea = satisfaccion_aeropuerto.groupby('airportname')['satisfaction'].value_counts(normalize=True).unstack()

# Por ultimo mostramos la proporción de pasajeros satisfechos para cada aerolínea
print(satisfaccion_por_aerolinea['satisfied'])

airportname
28 de Noviembre Airport                            0.388889
Aasiaat Airport                                    0.372093
Aba Tenna Dejazmach Yilma International Airport    0.312500
Abau Airport                                       0.457143
Acadiana Regional Airport                          0.514286
                                                     ...   
Zhangjiakou Ningyuan Airport                       0.515152
Zhengzhou Xinzheng International Airport           0.511111
Île des Pins Airport                               0.315789
Öndörkhaan Airport                                 0.326087
Ørland Airport                                     0.500000
Name: satisfied, Length: 943, dtype: float64


In [44]:
# Para visualizar mejor los resultados colocamos las proporciones de satisfacción en orden descendente
satisfaccion_ordenada = satisfaccion_por_aerolinea['satisfied'].sort_values(ascending=False)

# Podemos visualizar las 10 aerolíneas con la mayor proporción de pasajeros satisfechos
print("Aerolíneas con la mayor proporción de pasajeros satisfechos:")
print(satisfaccion_ordenada.head(10))

# Tambien visualizamos las 10 aerolíneas con la menor proporción de pasajeros satisfechos
print("\nAerolíneas con la menor proporción de pasajeros satisfechos:")
print(satisfaccion_ordenada.tail(10))

Aerolíneas con la mayor proporción de pasajeros satisfechos:
airportname
Boalang Airport                      0.707317
Pangborn Memorial Airport            0.680851
Vanrook Station Airport              0.676471
Balurghat Airport                    0.676471
John H Batten Airport                0.675000
Bata Airport                         0.674419
La Tontouta International Airport    0.666667
Héroes De Malvinas Airport           0.631579
Sui Airport                          0.625000
Tanjung Harapan Airport              0.625000
Name: satisfied, dtype: float64

Aerolíneas con la menor proporción de pasajeros satisfechos:
airportname
Dillon's Bay Airport                     0.250000
Rogers Municipal Airport-Carter Field    0.250000
Yampa Valley Airport                     0.243243
Clarks Point Airport                     0.243243
London Biggin Hill Airport               0.241379
Enarotali Airport                        0.236842
Maota Airport                            0.229167
Blytheville

#### 5. ¿La satisfacción de los pasajeros difiere según el tipo de viaje o la clase de viaje?

In [45]:
# Combinamos las tablas 'viaje_satisfaccion' y 'viaje'
satisfaccion_viaje = pd.merge(df_viaje_satisfaccion, df_viajes, left_on='idviaje', right_on='idviaje')

# Calculamos la proporción de pasajeros satisfechos para cada tipo de viaje y clase de viaje
satisfaccion_por_tipo = satisfaccion_viaje.groupby('travel_type')['satisfaction'].value_counts(normalize=True).unstack()
satisfaccion_por_clase = satisfaccion_viaje.groupby('class')['satisfaction'].value_counts(normalize=True).unstack()

# Visualizamos la proporción de pasajeros satisfechos para cada tipo de viaje y clase de viaje
print("Satisfacción por tipo de viaje:")
print(satisfaccion_por_tipo['satisfied'])

print("\nSatisfacción por clase de viaje:")
print(satisfaccion_por_clase['satisfied'])

Satisfacción por tipo de viaje:
travel_type
Business travel    0.578555
Personal Travel    0.103838
Name: satisfied, dtype: float64

Satisfacción por clase de viaje:
class
Business    0.690249
Eco         0.186575
Eco Plus    0.239009
Name: satisfied, dtype: float64
